In [4]:
df = spark.read.table("`merged_customer_silver`")


StatementMeta(, 51dd1a2b-ec8b-45de-89cc-bc1deb8af874, 6, Finished, Available, Finished)

# Creating date dimension Table

In [11]:
from pyspark.sql.types import *
from delta.tables import*

# # Define the schema for the dimdate_gold table
DeltaTable.createIfNotExists(spark) \
.tableName("Customer_dimdate_gold") \
.addColumn("JoinDate", DateType()) \
.addColumn("LeaveDate")
.addColumn("DateID", LongType())\
.addColumn("Day", LongType()) \
.addColumn("Month", LongType()) \
.addColumn("Year", LongType()) \
.execute()

StatementMeta(, 34ab4b8f-7648-4caf-9403-aebc83eef525, 13, Finished, Available, Finished)

In [19]:
from pyspark.sql.functions import col, dayofmonth, month, year, date_format

# # Create dataframe for dimDate_gold

dfCustomer_dimDate_gold = df.dropDuplicates(["Join_Date"]).select(col("Join_Date"), 
col("Year"), \
col("Month"), \
dayofmonth("Join_Date").alias("Day"), \
).orderBy("Year")

# # Display the first 10 rows of the dataframe to preview your data

display(dfCustomer_dimDate_gold.head(10))

StatementMeta(, 34ab4b8f-7648-4caf-9403-aebc83eef525, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9c3780dd-ccd2-4f49-b74e-3bc35fdfc46a)

# Create Date ID 

In [42]:
from pyspark.sql.functions import monotonically_increasing_id, col, when, coalesce, max, lit

dfdimDate_temp = spark.read.table("Customer_dimDate_gold")

MAXDateID = dfdimDate_temp.select(coalesce(max(col("DateID")),lit(0)).alias("MAXDateID")).first()[0]

dfCustomer_dimDate_gold = dfCustomer_dimDate_gold.join(dfdimDate_temp,(dfCustomer_dimDate_gold.Join_Date == dfdimDate_temp.Join_Date), "left_anti")

dfCustomer_dimDate_gold = dfCustomer_dimDate_gold.withColumn("DateID",monotonically_increasing_id() + MAXDateID + 1)

# # Display the first 10 rows of the dataframe to preview your data

display(dfCustomer_dimDate_gold.head(10))

StatementMeta(, 34ab4b8f-7648-4caf-9403-aebc83eef525, 44, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 874a8b58-7273-4f86-914f-6637843d99c3)

In [25]:
dfCustomer_dimDate_gold.write.format("delta").mode("overwrite").saveAsTable("NCustomer_dimDate_gold")

StatementMeta(, 34ab4b8f-7648-4caf-9403-aebc83eef525, 27, Finished, Available, Finished)

# Creating Region Dimension

In [26]:
from pyspark.sql.types import *
from delta.tables import *

# # Create customer_gold dimension delta table
DeltaTable.createIfNotExists(spark) \
.tableName("Customer_dimRegion_gold") \
.addColumn("Province", StringType()) \
.addColumn("City", StringType()) \
.addColumn("RegionID", LongType()) \
.execute()

StatementMeta(, 34ab4b8f-7648-4caf-9403-aebc83eef525, 28, Finished, Available, Finished)

In [27]:
from pyspark.sql.functions import col, dayofmonth, month, year, date_format

# # Create dataframe for dimDate_gold

dfCustomer_dimRegion_gold = df.dropDuplicates(["Province"]).select(col("Province"), \
col("City"), 
).orderBy("Year")

# # Display the first 10 rows of the dataframe to preview your data

display(dfCustomer_dimRegion_gold.head(10))

StatementMeta(, 34ab4b8f-7648-4caf-9403-aebc83eef525, 29, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3f0fed63-3b87-452d-a44b-e903e0db2800)

# Create region ID 

In [40]:
from pyspark.sql.functions import monotonically_increasing_id, col, when, coalesce, max, lit

dfdimRegion_temp = spark.read.table("Customer_dimRegion_gold")

MAXRegionID = dfdimRegion_temp.select(coalesce(max(col("RegionID")),lit(0)).alias("MAXRegionID")).first()[0]

dfCustomer_dimRegion_gold = dfCustomer_dimRegion_gold.join(dfdimRegion_temp,(dfCustomer_dimRegion_gold.Province == dfdimRegion_temp.Province), "left_anti")

dfCustomer_dimRegion_gold = dfCustomer_dimRegion_gold.withColumn("RegionID",monotonically_increasing_id() + MAXRegionID + 1)

# # Display the first 10 rows of the dataframe to preview your data


display(dfCustomer_dimRegion_gold.head(10))

StatementMeta(, 34ab4b8f-7648-4caf-9403-aebc83eef525, 42, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a87eb0a1-d6ef-4315-a573-99fede5a2b85)

In [29]:
dfCustomer_dimRegion_gold.write.format("delta").mode("overwrite").saveAsTable("Customer_dimRegion_gold")

StatementMeta(, 34ab4b8f-7648-4caf-9403-aebc83eef525, 31, Finished, Available, Finished)

# Creating Demography dimension table

In [30]:
from pyspark.sql.types import *
from delta.tables import *

# # Create customer_gold dimension delta table
DeltaTable.createIfNotExists(spark) \
.tableName("Demo_dimRegion_gold") \
.addColumn("Postal_Code", StringType()) \
.addColumn("Gender", StringType()) \
.addColumn("Education", StringType())\
.addColumn("Marital_Status", StringType())\
.addColumn("DemoID", LongType()) \
.execute()

StatementMeta(, 34ab4b8f-7648-4caf-9403-aebc83eef525, 32, Finished, Available, Finished)

In [31]:
from pyspark.sql.functions import col, dayofmonth, month, year, date_format

# # Create dataframe for dimDate_gold

dfDemo_dimRegion_gold = df.dropDuplicates(["Province"]).select(col("Postal_Code"), \
col("Gender"), \
col("Education"), \
col("Marital_Status") 
).orderBy("Year")

# # Display the first 10 rows of the dataframe to preview your data

display(dfDemo_dimRegion_gold.head(10))

StatementMeta(, 34ab4b8f-7648-4caf-9403-aebc83eef525, 33, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d718fecb-e65f-4859-8c26-b29c5042fa71)

In [33]:
from pyspark.sql.functions import monotonically_increasing_id, col, when, coalesce, max, lit

dfdimRegion_temp = spark.read.table("Demo_dimRegion_gold")

MAXDemoID = dfdimRegion_temp.select(coalesce(max(col("DemoID")),lit(0)).alias("MAXDemoID")).first()[0]

dfDemo_dimRegion_gold = dfDemo_dimRegion_gold.join(dfdimRegion_temp,(dfDemo_dimRegion_gold.Education == dfdimRegion_temp.Education), "left_anti")

dfDemo_dimRegion_gold= dfDemo_dimRegion_gold.withColumn("DemoID",monotonically_increasing_id() + MAXDemoID + 1)

# # Display the first 10 rows of the dataframe to preview your data


display(dfDemo_dimRegion_gold.head(10))

StatementMeta(, 34ab4b8f-7648-4caf-9403-aebc83eef525, 35, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5f285f5a-af86-4833-9c7b-6950dad648b1)

In [34]:
dfDemo_dimRegion_gold.write.format("Delta").mode("overwrite").saveAsTable("Demo_dimRegion_gold")

StatementMeta(, 34ab4b8f-7648-4caf-9403-aebc83eef525, 36, Finished, Available, Finished)

# Creating Fact customer Table

In [5]:
from pyspark.sql.types import *
from delta.tables import *

DeltaTable.createIfNotExists(spark) \
.tableName("factCustomer_gold") \
.addColumn("Loyalty_Number", StringType())\
.addColumn("RegionID", LongType()) \
.addColumn("DateID", IntegerType())\
.addColumn("DemoID", LongType())\
.addColumn("Total_Flights", IntegerType())\
.addColumn("Distance", IntegerType())\
.addColumn("Points_Accumulated", IntegerType())\
.addColumn("Points_Redeemed", LongType())\
.addColumn("Dollar_Cost_Points_Redeemed", LongType())\
.addColumn("Points_Balance", IntegerType())\
.addColumn("Average_Distance_Per_Flight", IntegerType())\
.addColumn("Points_Per_Mile", IntegerType())\
.addColumn("Salary", IntegerType())\
.addColumn("Membership_Tier", StringType())\
.addColumn("Customer_Lifetime_Value", IntegerType())\
.addColumn("Enrollment_Type", StringType())\
.addColumn("Status", StringType())\
.execute()

StatementMeta(, 51dd1a2b-ec8b-45de-89cc-bc1deb8af874, 7, Finished, Available, Finished)

In [6]:
from pyspark.sql.functions import monotonically_increasing_id, col, when, coalesce, max, lit
dfdimRegion_temp = spark.read.table("Customer_dimRegion_gold")
dfdimDate_temp = spark.read.table("NCustomer_dimDate_gold")
dfdimDemo_temp = spark.read.table("Demo_dimRegion_gold")

dffactCustomer_gold = df.alias("df1") \
    .join(dfdimDate_temp.alias("df2"), col("df1.Month") == col("df2.Month"), "left") \
    .join(dfdimRegion_temp.alias("df3"), col("df1.Province") == col("df3.Province"), "left") \
    .join(dfdimDemo_temp.alias("df4"), col("df1.Postal_Code") == col("df4.Postal_Code")) \
    .select(
        col("df2.DateID"),
        col("df3.RegionID"),
        col("df4.DemoID"),
        col("Loyalty_Number"),
        col("df1.Total_Flights"),
        col("df1.Distance"),
        col("df1.Points_Accumulated"),
        col("df1.Points_Redeemed"),
        col("df1.Dollar_Cost_Points_Redeemed"),
        col("df1.Points_Balance"),
        col("df1.Average_Distance_Per_Flight"),
        col("df1.Points_Per_Mile"),
        col("df1.Salary"),
        col("df1.Membership_Tier"),
        col("df1.Customer_Lifetime_Value"),
        col("df1.Enrollment_Type"),
        col("Status")
    ).orderBy(col("df1.Year"), col("df2.DateID"), col("df3.RegionID"), col("df4.DemoID"))
display(dffactCustomer_gold.head(10))

StatementMeta(, 51dd1a2b-ec8b-45de-89cc-bc1deb8af874, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cdb6c493-1ecf-49f3-b79b-529ac7858443)

In [9]:
dffactCustomer_gold.write.format("delta").mode("overwrite").saveAsTable("NEW_FACT_CUSTOMER")

StatementMeta(, 51dd1a2b-ec8b-45de-89cc-bc1deb8af874, 11, Finished, Available, Finished)

# Editing my code